In this project we are trying to detect the stress based on the reddit post.In this we have used the bert-base transformer model and we have fine tuned it for getting the results.

In [ ]:
  import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dreaddit-train.csv')

In [ ]:
df = df[['text', 'label']]

In [ ]:
df.head()

,text,label
0,"He said he had not felt that way before, sugge...",1
1,"Hey there r/assistance, Not sure if this is th...",0
2,My mom then hit me with the newspaper and it s...,1
3,"until i met my new boyfriend, he is amazing, h...",1
4,October is Domestic Violence Awareness Month a...,1


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, TFAutoModel
import numpy as np

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name, max_length=70, add_special_tokens = True,padding='max_length', truncation=True, return_token_type_ids=False,return_tensors = 'tf')

In [ ]:
train = tokenizer(list(df['text'].values), return_tensors='tf', truncation=True, padding=True, max_length=70)

In [ ]:
y = np.asarray(df['label']).astype('float32')

In [ ]:
def tokenize(data):
  return  tokenizer(data,
                    add_special_tokens = True,
                    padding='max_length', 
                    truncation=True, 
                    max_length=70, 
                    return_token_type_ids=False,
                    return_tensors = 'tf')

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D, Dropout
from tensorflow.keras import Model

In [ ]:
bertnew = TFAutoModel.from_pretrained(model_name)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
bertnew

In [ ]:
input1 = Input(shape=(70, ), dtype=tf.int32, name='input_ids')
input2 = Input(shape=(70, ), dtype=tf.int32, name='attention_mask')

bertnew = bertnew(input1, input2)[0]
output = GlobalAveragePooling1D()(bertnew)
output = Dense(128, activation='relu')(output)
output = Dropout(0.3)(output)
output = Dense(1, activation='sigmoid')(output)

In [ ]:
model = Model(inputs=[input1, input2], outputs=output)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit([train['input_ids'],  train['attention_mask']], y, epochs=9, batch_size=64)

Epoch 1/9


45/45 [==============================] - 107s 800ms/step - loss: 0.5226 - accuracy: 0.7347
Epoch 2/9
45/45 [==============================] - 37s 823ms/step - loss: 0.3361 - accuracy: 0.8510
Epoch 3/9
45/45 [==============================] - 38s 837ms/step - loss: 0.1477 - accuracy: 0.9479
Epoch 4/9
45/45 [==============================] - 38s 853ms/step - loss: 0.0791 - accuracy: 0.9715
Epoch 5/9
45/45 [==============================] - 39s 860ms/step - loss: 0.0250 - accuracy: 0.9926
Epoch 6/9
45/45 [==============================] - 39s 869ms/step - loss: 0.0433 - accuracy: 0.9831
Epoch 7/9
45/45 [==============================] - 40s 881ms/step - loss: 0.0184 - accuracy: 0.9940
Epoch 8/9
45/45 [==============================] - 39s 873ms/step - loss: 0.0104 - accuracy: 0.9968
Epoch 9/9
45/45 [==============================] - 40s 878ms/step - loss: 0.0058 - accuracy: 0.9979
